In [1]:
import pandas as pd
import pandas as pd
import string
import re

In [3]:
with open('businessCard.tsv',mode='r',encoding='utf8',errors='ignore') as f:
    text=f.read()

In [8]:
data=list(map(lambda x:x.split('\t'),text.split('\n')))

In [49]:
df=pd.DataFrame(data[1:],columns=data[0])

In [50]:
df.dropna(inplace=True)

In [51]:
df

,id,text,tag
0,000.jpeg,,O
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"8881,",I-PHONE
4,000.jpeg,90309,B-PHONE
...,...,...,...
10440,290.jpeg,,O
10441,290.jpeg,Richard,B-NAME
10442,290.jpeg,Pretorius,I-NAME
10443,290.jpeg,,O


In [52]:
#Cleaning Text

In [53]:
whitespace=string.whitespace
punctuation='!"#$%&\'()*+-.:;<=>?[\\]^_`{|}~'

In [54]:
df

,id,text,tag
0,000.jpeg,,O
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"8881,",I-PHONE
4,000.jpeg,90309,B-PHONE
...,...,...,...
10440,290.jpeg,,O
10441,290.jpeg,Richard,B-NAME
10442,290.jpeg,Pretorius,I-NAME
10443,290.jpeg,,O


In [55]:
whitespace = string.whitespace
punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
def cleanText(txt):
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [56]:
df['text']=df['text'].apply(cleanText)

In [57]:
df

,id,text,tag
0,000.jpeg,,O
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"8881,",I-PHONE
4,000.jpeg,90309,B-PHONE
...,...,...,...
10440,290.jpeg,,O
10441,290.jpeg,richard,B-NAME
10442,290.jpeg,pretorius,I-NAME
10443,290.jpeg,,O


In [59]:
dataClean=df.query("text!= ''")

In [61]:
dataClean.head(10)

,id,text,tag
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"8881,",I-PHONE
4,000.jpeg,90309,B-PHONE
5,000.jpeg,52549,I-PHONE
6,000.jpeg,fi,O
7,000.jpeg,/laurelsoverseaseducation,O
8,000.jpeg,���@,O
9,000.jpeg,laurels,B-ORG
10,000.jpeg,overseas,I-ORG


In [62]:
group=dataClean.groupby('id')

In [85]:
card=group.groups.keys()

In [86]:
allcardsdata=[]

In [89]:
for c in card:
    c_data=[]
    grouparray=group.get_group(c)[['text','tag']].values
    content=''
    annotations={'entities':[]}
    start=0
    end=0
    for text,label in grouparray:
        text=str(text)
        stringlen=len(text)+1

        start=end
        end=start+stringlen

        if label!='O':
            annot=(start,end-1,label)
            annotations['entities'].append(annot)
        content=content+ text+' '
    c_data=(content,annotations)
    allcardsdata.append(c_data)

In [90]:
allcardsdata

[('. 040-4852 8881, 90309 52549 fi /laurelsoverseaseducation ���@ laurels overseas educational consultancy pvt. ltd. sea u.k australia canada ireland www.laurelseducation.com info@laurelseducation.com ',
  {'entities': [(2, 10, 'B-PHONE'),
    (11, 16, 'I-PHONE'),
    (17, 22, 'B-PHONE'),
    (23, 28, 'I-PHONE'),
    (63, 70, 'B-ORG'),
    (71, 79, 'I-ORG'),
    (80, 91, 'I-ORG'),
    (92, 103, 'I-ORG'),
    (104, 108, 'I-ORG'),
    (109, 113, 'I-ORG'),
    (147, 171, 'B-WEB'),
    (172, 197, 'B-EMAIL')]}),
 ('john smith marketing manager web www.psdgraphics.com phone 123-456-7890 mail email@psdgraphics.com ',
  {'entities': [(0, 4, 'B-NAME'),
    (5, 10, 'I-NAME'),
    (11, 20, 'B-DES'),
    (21, 28, 'I-DES'),
    (33, 52, 'B-WEB'),
    (59, 71, 'B-PHONE'),
    (77, 98, 'B-EMAIL')]}),
 ('sau 0 98489 24441 dy 08672, 224441 /enkateswapa wie ',
  {'entities': [(6, 11, 'B-PHONE'), (12, 17, 'I-PHONE'), (35, 47, 'B-ORG')]}),
 ('prasad @ 9,96,31,73,53,59,49,04,00,000 i flex design album desi

In [92]:
#Split the data
import random
random.shuffle(allcardsdata)

In [93]:
TrainData=allcardsdata[:240]
TestData=allcardsdata[240:]

In [95]:
import pickle
pickle.dump(TrainData,open('./data/TrainData.pickle',mode='wb'))
pickle.dump(TestData,open('./data/TestData.pickle',mode='wb'))